<div style="background-color:black;color:white;padding:20px">

<center><h3> LAA </h3></center>
<center><h1> Projeto Bicóptero </h1></center>

<center> <h1> Modelo dinâmico da bancada usando a formulação de Lagrange</h1></center>

<h3> Por Gabriel Henrique Costa e Silva </h3> 
</div>

<h2> Inicializando as bibliotecas </h2>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import * 
from sympy import trigsimp, pi, cos, sin
from sympy.physics.mechanics import *

# Setup

First, import the necessary functions from SymPy that will allow us to construct time varying vectors in the four reference frames.

In [2]:
from __future__ import print_function, division
from sympy import symbols, simplify
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point

SymPy has a rich printing system. Here we initialize printing so that all of the mathematical equations are rendered in standard mathematical notation.

In [3]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

The IPython notebook can display rich content. We will use the Image function to import some images for reference.

In [4]:
from IPython.display import Image

In [5]:
t = Symbol("t")

# Angulos Theta
theta1 = Function("theta1")(t)
theta2 = Function("theta2")(t)
theta3 = Function("theta3")(t)
theta4 = Function("theta4")(t)

# Constante da gravidade
g = dynamicsymbols('g')

# Forcas e Momentos
fx, fy, fz = dynamicsymbols('f_x, f_y, f_z')
taux, tauy, tauz = dynamicsymbols('tau_x, tau_y, tau_z')

# Forcas dos motores
f1,f2= dynamicsymbols('F_1, F_2')

# Angulos Alfa (Angulos do mecanismo tilt)
alpha1 = Function("alpha1")(t)
alpha2 = Function("alpha2")(t)

# Extensão dos bracos do bicopter
b1,b2 = dynamicsymbols('b_1, b_2')

In [6]:
l0, l1, l2, l3, l4 = dynamicsymbols('l0, l1, l2, l3, l4')

In [7]:
m1, m2, m3, m4, m5 = dynamicsymbols('m1, m2, m3, m4, m5')

In [8]:
Ix1, Ix2, Ix3, Ix4, Ix5 = dynamicsymbols('Ix1, Ix2, Ix3, Ix4, Ix5')
Iy1, Iy2, Iy3, Iy4, Iy5 = dynamicsymbols('Iy1, Iy2, Iy3, Iy4, Iy5')
Iz1, Iz2, Iz3, Iz4, Iz5 = dynamicsymbols('Iz1, Iz2, Iz3, Iz4, Iz5')

In [9]:
d_theta1, d_theta2, d_theta3, d_theta4 = dynamicsymbols('delta_theta_1, delta_theta_2, delta_theta_3, delta_theta_4')


In [10]:
d_omega1 = Function("delta_omega_1")(t)
d_omega2 = Function("delta_omega_2")(t)
d_omega3 = Function("delta_omega_3")(t)
d_omega4 = Function("delta_omega_4")(t)
omega10, omega20, omega30, omega40 = dynamicsymbols('omega1_0, omega2_0, omega3_0, omega4_0')

In [11]:
d_psi1 = Function("delta_psi_1")(t)
d_psi3 = Function("delta_psi_2")(t)
d_psi2 = Function("delta_psi_3")(t)
d_psi4 = Function("delta_psi_4")(t)
psi1, psi2, psi3, psi4 = dynamicsymbols('psi_1, psi_2, psi_3, psi_4')
sigma = dynamicsymbols('sigma')
psi10, psi20, psi30, psi40 = dynamicsymbols('psi1_0, psi2_0, psi3_0, psi4_0')

In [12]:
theta10, theta20, theta30, theta40 = dynamicsymbols('theta_1_0, theta2_0, theta3_0, theta4_0')
alpha10, alpha20= dynamicsymbols('alpha_1_0, alpha2_0')
fx0, fy0, fz0 = dynamicsymbols('f_x0, f_y0, f_z0')
f10, f20= dynamicsymbols('F_10, F_20')
taux0, tauy0, tauz0 = dynamicsymbols('tau_x0, tau_y0, tau_z0')
d_fx, d_fy, d_fz, d_tauz = dynamicsymbols('delta_fx, delta_fy, delta_fz, delta_tau_z')

In [13]:
# Criando a função de criação da matriz de transformação homogênea

# Matriz Rotação da Transformação
def rt_M(T):
    M = T[0:3,0:3]
    return M

In [14]:
# Matriz Rotação da Transformação
def ch_M(T):
    M=zeros(6)
    M[0:3,0:3] = T[0:3,0:3]
    M[3:6,3:6] = T[0:3,0:3]
    return M

In [15]:
# Matriz da massa e inércia
def mI_M(m,Ix,Iy,Iz):
    M = diag(m, m, m, Ix, Iy, Iz)
    return M

In [16]:
# Ponto de representação homogenea para dimensão 3
def ph_2N(ph):
    p = Matrix(ph[0:3])
    return (p)

In [17]:
# Definindo a matriz jacobiana
def Jc(j1,j2,j3,j4):
    J = zeros(6,1)
    J = J.col_insert(0,j1)
    J = J.col_insert(1,j2)
    J = J.col_insert(2,j3)
    J = J.col_insert(3,j4)
    J.col_del(4)
    return J

In [18]:
# Definindo a nesima coluna da matriz jacobiana
def Ji(m0,op,oi):
    aux1 = m0*Matrix([0,0,1])
    aux2 = aux1.cross(op-oi)
    J = Matrix([aux2, aux1])
    return J

In [19]:
# Definindo a nesima coluna da matriz jacobiana
def JpTi(m0,op,Ti):
    # Calculate the zi= i-1Mi.k
    aux1 = m0*Matrix([0,0,1])
    oi = Matrix([Ti[0:3,3]])
    # Calculate the zi= i-1Mi.k
    aux2 = aux1.cross(op-oi)
    
    J = Matrix([aux2, aux1])
    return J

In [20]:
# Criando a função de criação da matriz de transformação homogênea
# a partir dos parâmetros de Denavit-Hatemberg

#cos = cos
#sin = sin

# Matriz Transformação Homogênea (Homogeous Transformation Matrix)
# Using the Standard Denavit-Hatemberg Convention (not Modified)
def ht_M(theta, d, alpha, a):
    # theta e alpha em radianos
    # d e a em metros
    Rz = Matrix([[cos(theta), -sin(theta), 0, 0],
                   [sin(theta), cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, a],
                   [0, 0, 0, 1]])
    ta = Matrix([[1, 0, 0, d],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = Matrix([[1, 0, 0, 0],
                   [0, cos(alpha), -sin(alpha), 0],
                   [0, sin(alpha), cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

<h2> Declaração dos parâmetros de Denavit-Hatenberg </h2>
<h2> Instanciação das matrizes de transformação homogêneas </h2>

![title](img/ModeloModificado.png)

In [21]:
T1 = ht_M(theta1, 0, (pi)/2, l1)
T2 = ht_M(theta2, l2, 0, 0)
T3 = ht_M(theta3, l3, (-pi)/2, 0)
T4 = ht_M(theta4, l4, 0, 0)

In [22]:
T1

Matrix([
[cos(theta1), 0,  sin(theta1),  0],
[sin(theta1), 0, -cos(theta1),  0],
[          0, 1,            0, l1],
[          0, 0,            0,  1]])

In [23]:
T2

Matrix([
[cos(theta2), -sin(theta2), 0, l2*cos(theta2)],
[sin(theta2),  cos(theta2), 0, l2*sin(theta2)],
[          0,            0, 1,              0],
[          0,            0, 0,              1]])

In [24]:
T3

Matrix([
[cos(theta3),  0, -sin(theta3), l3*cos(theta3)],
[sin(theta3),  0,  cos(theta3), l3*sin(theta3)],
[          0, -1,            0,              0],
[          0,  0,            0,              1]])

In [25]:
T4

Matrix([
[cos(theta4), -sin(theta4), 0, l4*cos(theta4)],
[sin(theta4),  cos(theta4), 0, l4*sin(theta4)],
[          0,            0, 1,              0],
[          0,            0, 0,              1]])

<h3> Simplificação da matriz T3 a partir da relação entre $\theta_3$ e $\theta_2$</h3>

In [26]:
T3

Matrix([
[cos(theta3),  0, -sin(theta3), l3*cos(theta3)],
[sin(theta3),  0,  cos(theta3), l3*sin(theta3)],
[          0, -1,            0,              0],
[          0,  0,            0,              1]])

In [27]:
T3=T3.subs(theta3,(-pi/2-theta2))

In [28]:
T3

Matrix([
[-sin(theta2),  0,  cos(theta2), -l3*sin(theta2)],
[-cos(theta2),  0, -sin(theta2), -l3*cos(theta2)],
[           0, -1,            0,               0],
[           0,  0,            0,               1]])

<h2> Instanciação das matrizes de transformação homogêneas em realação ao referencial 0 </h2>

In [29]:
T01=T1
T02=T01*T2
T02=trigsimp(T02)
T03=T02*T3
T03=trigsimp(T03)
T04=T03*T4

In [30]:
#(T02)
#(T03)
#(T04)

<h2> Criação das matrizes Jacobianas</h2>

In [31]:
# Criação dos pontos origens
og1=ph_2N(T01*Matrix([0,0,0,1]))
og2=ph_2N(T03*Matrix([-l3/2,0,0,1]))
og3=ph_2N(T03*Matrix([0,0,0,1]))
og4=ph_2N(T04*Matrix([0,0,0,1]))

In [32]:
# Calculo do Jc[1] que é o jacobiano referente ponto centro de gravidade g1
null=Matrix([0,0,0,0,0,0])
J1 = Ji(eye(3),og1, Matrix([0,0,0])) 
Jc1 = Jc(J1,null,null,null)
# Calculo do Jc[2] que é o jacobiano referente ponto centro de gravidade g2
J1 = Ji(eye(3),og2, Matrix([0,0,0])) 
J2 = Ji(rt_M(T01),og2, Matrix([0,0,0])) 
Jc2 = trigsimp(Jc(J1,J2,null,null))
# Calculo do Jc[3] que é o jacobiano referente ponto centro de gravidade g3
J1 = Ji(eye(3),og3, Matrix([0,0,0])) 
J2 = Ji(rt_M(T01),og3, Matrix([0,0,0]))
J3 = JpTi(rt_M(T02),og3,T02) 
Jc3 = trigsimp(Jc(J1,J2,J3,null))
# Calculo do Jc[4] que é o jacobiano referente ponto centro de gravidade g4
J1 = Ji(eye(3),og4, Matrix([0,0,0])) 
J2 = Ji(rt_M(T01),og4, Matrix([0,0,0]))
J3 = JpTi(rt_M(T02),og4,T02) 
J4 = JpTi(rt_M(T03),og4,T03)
Jc4 = trigsimp(Jc(J1,J2,J3,J4))

In [33]:
Jc1

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[1, 0, 0, 0]])

In [34]:
Jc2

Matrix([
[-l2*sin(theta1)*cos(theta2), -(l1 + l2*sin(theta2) - l3/2)*cos(theta1), 0, 0],
[ l2*cos(theta1)*cos(theta2), -(l1 + l2*sin(theta2) - l3/2)*sin(theta1), 0, 0],
[                          0,                            l2*cos(theta2), 0, 0],
[                          0,                               sin(theta1), 0, 0],
[                          0,                              -cos(theta1), 0, 0],
[                          1,                                         0, 0, 0]])

In [35]:
Jc3

Matrix([
[-l2*sin(theta1)*cos(theta2), -(l1 + l2*sin(theta2) - l3)*cos(theta1), l3*cos(theta1), 0],
[ l2*cos(theta1)*cos(theta2), -(l1 + l2*sin(theta2) - l3)*sin(theta1), l3*sin(theta1), 0],
[                          0,                          l2*cos(theta2),              0, 0],
[                          0,                             sin(theta1),    sin(theta1), 0],
[                          0,                            -cos(theta1),   -cos(theta1), 0],
[                          1,                                       0,              0, 0]])

In [36]:
Jc4

Matrix([
[-l2*sin(theta1)*cos(theta2) - l4*sin(theta4)*cos(theta1), -(l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*cos(theta1), (l3 + l4*cos(theta4))*cos(theta1), -l4*sin(theta1)*cos(theta4)],
[ l2*cos(theta1)*cos(theta2) - l4*sin(theta1)*sin(theta4), -(l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*sin(theta1), (l3 + l4*cos(theta4))*sin(theta1),  l4*cos(theta1)*cos(theta4)],
[                                                       0,                                           l2*cos(theta2),                                 0,              l4*sin(theta4)],
[                                                       0,                                              sin(theta1),                       sin(theta1),                 cos(theta1)],
[                                                       0,                                             -cos(theta1),                      -cos(theta1),                 sin(theta1)],
[                                                       1,                       

In [37]:
# Calculo do J
J = Jc(J1,J2,J3,J4)

In [38]:
# Definindo a matriz jacobiana
def Jc(j1,j2,j3,j4):
    J = zeros(6,1)
    J = J.col_insert(0,j1)
    J = J.col_insert(1,j2)
    J = J.col_insert(2,j3)
    J = J.col_insert(3,j4)
    J.col_del(4)
    return J

In [39]:
J

Matrix([
[-l2*sin(theta1)*cos(theta2) - l4*sin(theta4)*cos(theta1),                                                                                      -(l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*cos(theta1), -(-l3 - l4*cos(theta4))*cos(theta1),                                   -l4*sin(theta1)*cos(theta4)],
[ l2*cos(theta1)*cos(theta2) - l4*sin(theta1)*sin(theta4),                                                                                      -(l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*sin(theta1), -(-l3 - l4*cos(theta4))*sin(theta1),                                    l4*cos(theta1)*cos(theta4)],
[                                                       0, (l2*sin(theta1)*cos(theta2) + l4*sin(theta4)*cos(theta1))*sin(theta1) + (l2*cos(theta1)*cos(theta2) - l4*sin(theta1)*sin(theta4))*cos(theta1),                                   0, l4*sin(theta1)**2*sin(theta4) + l4*sin(theta4)*cos(theta1)**2],
[                                                       0,                        

<h2> Criação das matrizes de Inércia</h2>

In [40]:
mI_1 = mI_M(m1,Ix1,Iy1,Iz1)
mI_2 = mI_M(m2,Ix2,Iy2,Iz2)
mI_3 = mI_M(m3,Ix3,Iy3,Iz3)
mI_4 = mI_M(m4,Ix4,Iy4,Iz4)
mI_4

Matrix([
[m4,  0,  0,   0,   0,   0],
[ 0, m4,  0,   0,   0,   0],
[ 0,  0, m4,   0,   0,   0],
[ 0,  0,  0, Ix4,   0,   0],
[ 0,  0,  0,   0, Iy4,   0],
[ 0,  0,  0,   0,   0, Iz4]])

In [41]:
# Build the Di matriz
def Di_M(Jc_gi,mI):
    M = (Jc_gi.T)*mI*Jc_gi;
    return M

In [42]:

D1 = Di_M(Jc1,mI_1)
D2 = trigsimp(Di_M(Jc2,mI_2))
D3 = trigsimp(Di_M(Jc3,mI_3))
D4 = trigsimp(Di_M(Jc4,mI_4))
Dtotal = D1+D2+D3+D4

In [43]:
Dtotal

Matrix([
[Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2*cos(theta2)**2 + l2**2*m3*cos(theta2)**2 - l2**2*m4*sin(theta2)**2 + l2**2*m4 + l4**2*m4*sin(theta4)**2,                                                                                                                                                                                                                                                                                                                                                                                                                                         (l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*l4*m4*sin(theta4),                                                                                                                                                                                                                                                             -(l3 + l4*cos(theta4))*l4*m4*sin(theta4),                                             l2*l4*m4*cos(theta2)*cos(th

<h2> Calculo da Energia Cinética </h2>

In [44]:
q=Matrix([theta1,theta2,theta3,theta4])
q_dot= diff(q,t)

In [45]:
K = 1/2* q_dot.T * Dtotal * q_dot
K

Matrix([[(0.5*(Ix4*sin(2*theta1)/2 - Iy4*sin(2*theta1)/2 + l2*l4*m4*sin(theta4)*cos(theta2))*theta4' + 0.5*(l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*l4*m4*sin(theta4)*theta1' + 0.5*(Ix3*sin(theta1)**2 - Ix4*cos(theta1)**2 + Ix4 - Iy3*sin(theta1)**2 + Iy3 + Iy4*cos(theta1)**2 - l1*l3*m3 - l1*l3*m4 - l1*l4*m4*cos(theta4) - l2*l3*m3*sin(theta2) - l2*l3*m4*sin(theta2) - l2*l4*m4*sin(theta2)*cos(theta4) + l3**2*m3 + l3**2*m4 + 2*l3*l4*m4*cos(theta4) + l4**2*m4*cos(theta4)**2)*theta3' + 0.5*((l1 + l2*sin(theta2) - l3 - l4*cos(theta4))**2*m4*sin(theta1)**2 + (l1 + l2*sin(theta2) - l3 - l4*cos(theta4))**2*m4*cos(theta1)**2 + Ix2*sin(theta1)**2 + Ix3*sin(theta1)**2 + Ix4*sin(theta1)**2 - Iy2*sin(theta1)**2 + Iy2 - Iy3*sin(theta1)**2 + Iy3 + Iy4*cos(theta1)**2 + l1**2*m2 + l1**2*m3 + 2*l1*l2*m2*sin(theta2) + 2*l1*l2*m3*sin(theta2) - l1*l3*m2 - 2*l1*l3*m3 + l2**2*m2 + l2**2*m3 + l2**2*m4*cos(theta2)**2 - l2*l3*m2*sin(theta2) - 2*l2*l3*m3*sin(theta2) + l3**2*m2/4 + l3**2*m3)*theta2')*theta2' + (

<h2> Calculo da Energia Potencial</h2>

In [46]:
og1=T01*Matrix([0,0,0,1])
og2=T03*Matrix([-l3/2,0,0,1])
og3=T03*Matrix([0,0,0,1])
og4=T04*Matrix([0,0,0,1])

In [47]:
P1 = m1* g * Matrix([0,0,1,0]).T * og1
P2 = m2* g * Matrix([0,0,1,0]).T * og2
P3 = m3* g * Matrix([0,0,1,0]).T * og3
P4 = m4* g * Matrix([0,0,1,0]).T * og4

In [48]:
Ptotal = P1+ P2 + P3 + P4
Ptotal

Matrix([[(l1 + l2*sin(theta2) - l3)*g*m3 + (l1 + l2*sin(theta2) - l3/2)*g*m2 + (l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*g*m4 + g*l1*m1]])

<h2> Calculo do Lagrangiano </h2>

In [49]:
L= K-Ptotal

<h2> Calculo dos termos da equaçoes de Euler Lagrange </h2>

In [50]:
# Termo D(q) contribuicao inercial vale D_total ja calculado precedemente
Dtotal

Matrix([
[Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2*cos(theta2)**2 + l2**2*m3*cos(theta2)**2 - l2**2*m4*sin(theta2)**2 + l2**2*m4 + l4**2*m4*sin(theta4)**2,                                                                                                                                                                                                                                                                                                                                                                                                                                         (l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*l4*m4*sin(theta4),                                                                                                                                                                                                                                                             -(l3 + l4*cos(theta4))*l4*m4*sin(theta4),                                             l2*l4*m4*cos(theta2)*cos(th

In [51]:
q[1]
diff(Dtotal,q[1])*q_dot[1]

Matrix([
[(-2*l2**2*m2*sin(theta2)*cos(theta2) - 2*l2**2*m3*sin(theta2)*cos(theta2) - 2*l2**2*m4*sin(theta2)*cos(theta2))*theta2',                                                                                                                                                                                                                                                                         l2*l4*m4*sin(theta4)*cos(theta2)*theta2',                                                                                         0, -l2*l4*m4*sin(theta2)*cos(theta4)*theta2'],
[                                                                               l2*l4*m4*sin(theta4)*cos(theta2)*theta2', (2*(l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*l2*m4*sin(theta1)**2*cos(theta2) + 2*(l1 + l2*sin(theta2) - l3 - l4*cos(theta4))*l2*m4*cos(theta1)**2*cos(theta2) + 2*l1*l2*m2*cos(theta2) + 2*l1*l2*m3*cos(theta2) - 2*l2**2*m4*sin(theta2)*cos(theta2) - l2*l3*m2*cos(theta2) - 2*l2*l3*m3*cos(theta2))*theta2', 

In [52]:
def C_t(q,q_dot,D_total):
    C_1 = zeros(4,4)
    C_2 = zeros(4,1)

    for n in range(4):
        # Part 1 = dD(q)/dt = sum (partial D(q)/partial qi)q_dot
        C_1= C_1 + (diff(D_total,q[n])*q_dot[n])

    C_1 = C_1*q_dot

    for n in range(4):
        # Part 2 = column matrix (1/2)*(q_dot)^T.(partial D(q)/partial qi).q_do
        C_2[n] = (1/2)*((q_dot.T)*diff(D_total,q[n])*q_dot)

    C = C_1 - C_2;
    return C

In [53]:
# Termo C(q,qdot) contribuicao inercial

Ctotal = C_t(q, q_dot, Dtotal)
#Ctotal

In [54]:
def G_t(q,P_total):
    G = zeros(4,1)
    for n in range(4):
        G[n] = diff(P_total,q[n])
    return G

In [55]:
# Termo G(q)  vale as contribuicoes da gravidade 
Gtotal = G_t(q,Ptotal)
Gtotal

Matrix([
[                                                              0],
[g*l2*m2*cos(theta2) + g*l2*m3*cos(theta2) + g*l2*m4*cos(theta2)],
[                                                              0],
[                                            g*l4*m4*sin(theta4)]])

Calculo do esforcos de interaçao Gamma_e  



Representar os tensores cinematicos em relacao a P
1-Identificar o vetores OiP sendo P o ponto do referencial bicoptero
2-Multiplicar por i-1Mi (matriz de rotacao) 
3-fazer o produto vetorial entre Zi x (i-1Mi * OiP)
4-Montar o tensor


In [56]:
def v_i(M0,Tp,Ti):
    # Calculate the zi= i-1Mi.k
    aux1 = M0*Matrix([0,0,1])

    Op = Tp[0:3,3]
    Oi = Ti[0:3,3]

    # Cross Product  zi x (Op-Oi)
    aux2 = (aux1.cross(Op-Oi))
    
    # Building cinematic tensor for pivot joints
    
    v = Matrix([aux2, aux1])
    return v 

In [57]:
#Matriz rotacional 0^M_0
# Calculate the zi= i-1Mi.k 
J1 = v_i(eye(3), T04, zeros(4,4))

J2 = trigsimp(v_i(rt_M(T01), T04, T01))

J3 = v_i(rt_M(T02),T04,T02)

J4 = v_i(rt_M(T03),T04,T03)

JcP4 = Jc(J1,J2,J3,J4)

v = (JcP4*Matrix([0,1,-1,0])).subs({theta1: 0, theta2:0,theta4: 0})
#(JcP4.T)
#v
#(JcP4.T)*Matrix([-fx,0,0,0,0,0])


In [58]:
JcP4

Matrix([
[-l2*sin(theta1)*cos(theta2) - l4*sin(theta4)*cos(theta1), -(l2*sin(theta2) - l3 - l4*cos(theta4))*cos(theta1), -(-l3 - l4*cos(theta4))*cos(theta1),                                   -l4*sin(theta1)*cos(theta4)],
[ l2*cos(theta1)*cos(theta2) - l4*sin(theta1)*sin(theta4), -(l2*sin(theta2) - l3 - l4*cos(theta4))*sin(theta1), -(-l3 - l4*cos(theta4))*sin(theta1),                                    l4*cos(theta1)*cos(theta4)],
[                                                       0,                                      l2*cos(theta2),                                   0, l4*sin(theta1)**2*sin(theta4) + l4*sin(theta4)*cos(theta1)**2],
[                                                       0,                                         sin(theta1),                         sin(theta1),                                                   cos(theta1)],
[                                                       0,                                        -cos(theta1),                        -cos

Calcular o Gamma_e a partir da matriz Jacobiana cinematica em relacao a P

In [59]:
#Forca expressa no referencial 0 (Não é o que queremos)

#tau_e:=<<f[x],f[y],f[z],tau[x],tau[y],tau[z]>>:
tau_e = Matrix([fx,fy,0,0,0,tauz])
Gamma_e = JcP4.T*(tau_e)
Gamma_e.subs({theta2:0,theta1:0,theta4:0})


Matrix([
[ f_y*l2 + tau_z],
[-(-l3 - l4)*f_x],
[-(-l3 - l4)*f_x],
[         f_y*l4]])

In [60]:
# Para entao representar o vetor esforco em outro referencial é necessario as seguintes formulacoes

# Para entender indica-se o exemplo da maçã do link https://modernrobotics.northwestern.edu/nu-gm-book-resource/3-4-wrenches/#department
#Ttest:=eval(ht_M(0,-(ele),0,0));
#tau_e:=<<0,0,0,0,-mg,0>>;
#p:=v_2so3(<<Ttest(1,4),Ttest(2,4),Ttest(3,4)>>);
#T_adj(Ttest,rt_M(Ttest));
#tau_f:=LinearAlgebra[Transpose](T_adj(Ttest)).(tau_e);


#representacao de um vetor em so3
def v_2so3(v):
    p = zeros(3)
    p[0,1] = -v[2,0]
    p[0,2] = v[1,0]
    p[1,2] = -v[0,0]
    p[1,0] = v[2,0]
    p[2,0] = -v[1,0]
    p[2,1] = v[0,0]
    return p

#representacao adjunta da matriz Transformacao homogenea
def T_adj(Thom):
    p = v_2so3(Thom[0:3,3])

    K = p*rt_M(Thom)
    M = zeros(6)

    M[0:3,0:3] = Thom[0:3,0:3]
    M[3:6,0:6] = Matrix([K[0:3,0:3]]).col_insert(3,Thom[0:3,0:3]) 

    return M

#Inversa da matriz homogênea
def inv_Thom(Thom):
    Ka = (rt_M(Thom)).T
    Ke = -Ka*Thom[0:3,3] 

    M = Ka[0:3,0:3]
    M = M.col_insert(3,Ke[0:3,0])

    M = M.row_insert(4, Matrix([0,0,0,1]).T)
    return M

In [61]:
#Calcula-se a matrix inversa (T_0_4) da matriz transformacao homogenea T_04 
T40 =inv_Thom(T04)

#eval(T_4_[0],{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});

#Passa-se o vetor 3x1 p da entao matriz inversa para s03
p=v_2so3(T40[0:3,3])

#Finalmente calcula-se a matrix adjunta de T_4_0
MAdjT40 = T_adj(T40)
#T40
#p
#MAdjT40
#eval(AdjT_4_0,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});

#LinearAlgebra[Transpose](eval(AdjT_4_0,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0})).<<0,0,tau[z],f[x],f[y],0>> := LinearAlgebra[Transpose](eval(AdjT_4_0,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0})).<<0,0,tau[z],f[x],f[y],0>> ;

#Constroi-se uma matrix capaz de colocar os elementos momentos nas ultimas 3 posicoes do vetor 6x1 e colocar as forcas nas 3 primeiras posicoes do vetor
MF_M = zeros(6)
MF_M[0,3] =1
MF_M[1,4] =1
MF_M[2,5] =1
MF_M[3,0] =1
MF_M[4,1] =1
MF_M[5,2] =1


<h1> Mudando para o referencial do membro final <\h1>

In [62]:
#Perceba que inicialmente o vetor tau_e é da maneira <<momentos, forcas>>
# Alem disso para o nosso aplicacao taux=tauy=fz=0
tau_e = Matrix([0,0,tauz,fx,fy,0])

#Aqui mostra-se como os esforcos em P sao traduzidos no referencial 0
#eval(MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6),{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0}):
PJ4 = trigsimp(ch_M(rt_M(T40))*JcP4);

#aux:=MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6):

#LinearAlgebra[Transpose](J_cP[4]);

#Calcula-se Gamma_e
#Gamma_e:=LinearAlgebra[Transpose](J_cP[4]).MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6):

Gamma_e = PJ4.T*MF_M*(tau_e)

#Nosso caso queremos Gamma_b
Gamma_b = Gamma_e

Gamma_b = trigsimp(Gamma_b)
Gamma_b
#Visualização dos esforços em cada junta na posicao neutra 
#MF_M.<<0,0,tau[z],f[x],f[y],0>>;
#eval(MF_M.simplify(LinearAlgebra[Transpose](AdjT_4_0).(tau_e),eqtrig6),{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});
#eval(Gamma_b,{theta[2](t)=0,theta[1](t)=0,theta[4](t)=0});
PJ4.T

Matrix([
[ l2*sin(theta4)*cos(theta2), l2*cos(theta2)*cos(theta4),                       -l4*sin(theta4), -cos(theta4),  sin(theta4), 0],
[-l2*cos(theta2)*cos(theta4), l2*sin(theta4)*cos(theta2), -l2*sin(theta2) + l3 + l4*cos(theta4), -sin(theta4), -cos(theta4), 0],
[                          0,                          0,                   l3 + l4*cos(theta4), -sin(theta4), -cos(theta4), 0],
[                          0,                         l4,                                     0,            0,            0, 1]])

<h1> Traduzindo as variaveis de entrada com o mecanismo de vetorização<\h1>

![title](img/tiltmatrix.png)
![title](img/gammatilt.png)

In [63]:
tilt = Matrix([[-cos(alpha1), -cos(alpha2)],[sin(alpha1), -sin(alpha2)], zeros(3,2), [b1*cos(alpha1), -b2*cos(alpha2)]])
forces= Matrix([f1,f2])

In [64]:
#Calcula-se Gamma_e
Gamma_e = PJ4.T*tilt*forces

#Nosso caso queremos Gamma_b
Gamma_b = Gamma_e

Gamma_b = trigsimp(Gamma_b)
Gamma_b
Gamma_b.subs({theta1:0,theta2:0,theta4:0,alpha1:0,alpha2:0})

Matrix([
[                0],
[   (F_1 + F_2)*l2],
[                0],
[F_1*b_1 - F_2*b_2]])

<h1> Contra Peso no Robô <\h1>

In [65]:
Gamma_contra=zeros(4,1)
# Nao realizamos para o peso do elemento final que é o drone (por isso o range de 3)
Gamma_contra[0] = diff(P1,q[0])+diff(P2,q[0])+diff(P3,q[0])#+diff(P4,q[0])
Gamma_contra[1] = diff(P2,q[1])+diff(P3,q[1])#+diff(P4,q[1])
Gamma_contra[2] = diff(P3,q[2])#+diff(P4,q[2])
    
Gamma_contra

Matrix([
[                                        0],
[g*l2*m2*cos(theta2) + g*l2*m3*cos(theta2)],
[                                        0],
[                                        0]])

In [66]:
q=Matrix([theta1,theta2,theta3,theta4])
q_dot= diff(q,t)
q_ddot= diff(q_dot,t)

<h2> Equação Total do Movimento </h2>

In [67]:
Mdf=Dtotal*q_ddot+Ctotal+Gtotal-Gamma_b-Gamma_contra

<h2>Iniciando Equacao Solução</h2>

In [68]:
Eq=zeros(4,1)

In [69]:
#Isolando as variaveis q_ddot
Eq[0]=solve(Mdf[0], q_ddot[0])  #Relativo a Theta dois pontos 1
Eq[1]=solve(Mdf[1], q_ddot[1])  #Relativo a Theta dois pontos 2
Eq[2]=solve(Mdf[2], q_ddot[2])  #Relativo a Theta dois pontos 3
Eq[3]=solve(Mdf[3], q_ddot[3])  #Relativo a Theta dois pontos 4

<h2>Realizando a Linearização</h2>

<h2> Linearização (Método de Newton) </h2>

![title](img/linearizacao.png)

<a href="http://sites.poli.usp.br/d/PMR2400/3-%20Lineariza%C3%A7%C3%A3o%20de%20sistemas.pdf"> Material Referência </a>


<h2>Iniciando o vetor Estado e Comando<\h2>

In [70]:
states=Matrix([theta1, theta2, theta3, theta4])
inputs=Matrix([alpha1,alpha2,f1,f2])

In [71]:
len(inputs)

4

<h2>Iniciando o vetor estado inicial<\h2>

In [72]:
#Trim={fx:fx0,fy:fy0,fz:fz0,tauz:tauz0,theta1:theta1_0,theta2:theta2_0,theta3:-theta2_0,theta4:theta4_0,diff(theta1,t):omega10,diff(theta2,t):omega20,diff(theta3,t):omega30,diff(theta4,t):omega40,diff(diff(theta1,t),t):psi10,diff(diff(theta2,t),t):psi20,diff(diff(theta3,t),t):psi30,diff(diff(theta4,t),t):psi40}
#Trim_value={theta1_0: 0.00,theta2_0: 0.00,theta3_0: -(pi/2),theta4_0: 0.00,omega10: 0.00,omega20: 0.00,omega30: 0.00,omega40: 0.00,psi10: 0.00,psi20: 0.00,psi30: 0.00,psi40: 0.00}
Trim={f1:f10,f2:f20,alpha1:alpha10,alpha2:alpha20,theta1:theta10,theta2:theta20,theta3:-theta20,theta4:theta40,diff(theta1,t):omega10,diff(theta2,t):omega20,diff(theta3,t):omega30,diff(theta4,t):omega40,diff(diff(theta1,t),t):psi10,diff(diff(theta2,t),t):psi20,diff(diff(theta3,t),t):psi30,diff(diff(theta4,t),t):psi40}
states_value={theta10: 0.00,theta20: 0.00,theta30: -(pi/2),theta40: 0.00,omega10: 0.00,omega20: 0.00,omega30: 0.00,omega40: 0.00,psi10: 0.00,psi20: 0.00,psi30: 0.00,psi40: 0.00}
# inputs_value={alpha10: 0.00,alpha20: 0.00} #f10: g*m4/2,f20: g*m4/2}
inputs_value={alpha10: 0.00,alpha20: 0.00, f10: g*m4/2,f20: g*m4/2}

In [73]:
Eq_trim=zeros(4,1)

In [74]:
for i in range (len(inputs)):
#    Eq_trim[i]=Eq[i].subs(Trim).subs(states_value).subs(inputs_value)
    Eq_trim[i]=Eq[i].subs(Trim).subs(states_value).subs(inputs_value)

In [75]:
Eq_trim

Matrix([
[                                         0],
[                                         0],
[                                         0],
[(b_1*g*m4/2 - b_2*g*m4/2)/(Ix4 + l4**2*m4)]])

In [80]:
Eq_linha=Eq-Eq_trim
Eq_linha=Eq_linha.subs(Trim).subs(states_value)
Eq_linha

Matrix([
[                                                                                                                                                 (1.0*F_10*l2*sin(alpha_1_0) - 1.0*F_20*l2*sin(alpha2_0))/(Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2 + l2**2*m3 + l2**2*m4)],
[2.0*(2*F_10*l2*cos(alpha_1_0) + 2*F_20*l2*cos(alpha2_0) - 2.0*g*l2*m4)/(4.0*(l1 - l3 - l4)**2*m4 + 4.0*Iy2 + 4.0*Iy3 + 4.0*Iy4 + 4.0*l1**2*m2 + 4.0*l1**2*m3 - 4.0*l1*l3*m2 - 8.0*l1*l3*m3 + 4.0*l2**2*m2 + 4.0*l2**2*m3 + 4.0*l2**2*m4 + l3**2*m2 + 4.0*l3**2*m3)],
[                                                                                                                                                                                                                                                                 0],
[                                                                                                -(b_1*g*m4/2 - b_2*g*m4/2)/(Ix4 + l4**2*m4) + (F_10*b_1*cos(alpha_1_0) + F_10*l4*sin(alpha_1_0) - F_20*b_2*c

<h2>Iniciando Matrizes A e B<\h2>

In [98]:
A = zeros(4, 4) 
B = zeros(4, 4) 
inputs_value={alpha10: 0.00,alpha20: 0.00} #f10: g*m4/2,f20: g*m4/2}

In [104]:
for i in range(len(states)):
    for j in range(len(states)):
#        A[i,j]= simplify( ( diff( Eq_linha[i], states[j] ) ).subs(Trim).subs(states_value).subs(inputs_value))
        A[i,j]= simplify( ( diff( Eq[i], states[j] ) ).subs(Trim).subs(states_value).subs(inputs_value))
#        A[i,j]= simplify( ( diff( Eq[i], states[j] ) ).subs(Trim).subs(states_value) )

In [106]:
for i in range(len(states)):
    for j in range(len(inputs)):
#        B[i,j]= simplify( ( diff( Eq_linha[i], inputs[j] ) ).subs(Trim).subs(states_value).subs(inputs_value))
        B[i,j]= simplify( ( diff( Eq[i], inputs[j] ) ).subs(Trim).subs(states_value).subs(inputs_value) ) 
#        B[i,j]= simplify( ( diff( Eq[i], inputs[j] ) ).subs(Trim).subs(states_value) ) 

In [107]:
A

Matrix([
[0,                                                                                                                                                                                                                                                                                                                     0, 0, -1.0*(F_10 + F_20)*l2/(Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2 + l2**2*m3 + l2**2*m4)],
[0, 0.03125*(2*F_10 + 2*F_20 - 2.0*g*m4)*(8.0*(-l1 + l3 + l4)*m4 - 8.0*l1*m2 - 8.0*l1*m3 + 4.0*l3*m2 + 8.0*l3*m3)*l2**2/(0.5*(-l1 + l3 + l4)**2*m4 + 0.5*Iy2 + 0.5*Iy3 + 0.5*Iy4 + 0.5*l1**2*m2 + 0.5*l1**2*m3 - 0.5*l1*l3*m2 - l1*l3*m3 + 0.5*l2**2*m2 + 0.5*l2**2*m3 + 0.5*l2**2*m4 + 0.125*l3**2*m2 + 0.5*l3**2*m3)**2, 0,                                                                              0],
[0,                                                                                                                                                                                              

In [108]:
B

Matrix([
[1.0*F_10*l2/(Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2 + l2**2*m3 + l2**2*m4), -1.0*F_20*l2/(Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2 + l2**2*m3 + l2**2*m4),                                                                                                                                                                                                   0,                                                                                                                                                                                                   0],
[                                                                   0,                                                                     0, 4.0*l2/(4.0*(-l1 + l3 + l4)**2*m4 + 4.0*Iy2 + 4.0*Iy3 + 4.0*Iy4 + 4.0*l1**2*m2 + 4.0*l1**2*m3 - 4.0*l1*l3*m2 - 8.0*l1*l3*m3 + 4.0*l2**2*m2 + 4.0*l2**2*m3 + 4.0*l2**2*m4 + l3**2*m2 + 4.0*l3**2*m3), 4.0*l2/(4.0*(-l1 + l3 + l4)**2*m4 + 4.0*Iy2 + 4.0*Iy3 + 4.0*Iy4 + 4.0*l1**2*m2 + 4.0*l1**2*m3 - 4.0*l1*l3*m2 - 8.0*

In [109]:
delta_states=Matrix([(theta1-theta10), (theta2-theta20), (theta3-theta30), (theta4-theta40)])
delta_inputs=Matrix([(alpha1-alpha10), (alpha2-alpha20), (f1-f10), (f2-f20)])
Sol=A*delta_states+B*delta_inputs
Sol

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      -1.0*(F_10 + F_20)*(theta4 - theta4_0)*l2/(Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2 + l2**2*m3 + l2**2*m4) + 1.0*(alpha1 - alpha_1_0)*F_10*l2/(Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2 + l2**2*m3 + l2**2*m4) - 1.0*(alpha2 - alpha2_0)*F_20*l2/(Iz1 + Iz2 + Iz3 + Iz4 + l2**2*m2 + l2**2*m3 + l2**2*m4)],
[4.0*(F_1 - F_10)*l2/(4.0*(-l1 + l3 + l4)**2*m4 + 4.0*Iy2 + 4.0*Iy3 + 4.0*Iy4 + 4.0*l1**2*m2 + 4.0*l1**2*m3 - 4.0*l1*l3*m2 - 8.0*l1*l3*m3 + 4.0*l2**2*m2 + 4.0*l2**2*m3 + 4.0*l2**2*m4 + l3**2*m2 + 4.0*l3**2*m3) + 4.0*(F_2 - F_20)*l2/(4.0

<h2> Linearização (Método da pequena pertubação) </h2>

In [112]:
"""
d_psi1 = Function("delta_psi_1")(t)
d_psi3 = Function("delta_psi_2")(t)
d_psi2 = Function("delta_psi_3")(t)
d_psi4 = Function("delta_psi_4")(t)
psi1, psi2, psi3, psi4 = dynamicsymbols('psi_1, psi_2, psi_3, psi_4')
sigma = dynamicsymbols('sigma')
"""

'\nd_psi1 = Function("delta_psi_1")(t)\nd_psi3 = Function("delta_psi_2")(t)\nd_psi2 = Function("delta_psi_3")(t)\nd_psi4 = Function("delta_psi_4")(t)\npsi1, psi2, psi3, psi4 = dynamicsymbols(\'psi_1, psi_2, psi_3, psi_4\')\nsigma = dynamicsymbols(\'sigma\')\n'

In [113]:
#States:=[ seq( (theta[i](t) = theta[i[0]] + delta[theta[i]](t) ),i=1..4), seq( ( diff(theta[i](t),t) = omega[i[0]] + delta[omega[i]](t) ),i=1..4 ) ];
"""
states={theta1: theta1_0 + d_theta1, theta2: theta2_0 + d_theta2, theta3: theta3_0 + d_theta3, theta4: theta4_0 + d_theta4, diff(theta1): omega10+ d_omega1, diff(theta2): omega20+ d_omega2, diff(theta3): -omega20 - d_omega2, diff(theta4): omega40+ d_omega4} 
states
#States = [ seq( (theta[i](t) = theta[i[0]] + delta[theta[i]](t) ),i=1..4),seq( ( diff(theta[i](t),t) = omega[i[0]] + delta[omega[i]](t) ),i=1..2 ), diff(theta[3](t),t) = -omega_20 - deltaomega2,diff(theta[4](t),t) = omega[4[0]] + delta[omega[4]](t)  ];
inputs={fx:fx0+d_fx,fy:fy0+d_fy,fz:fz0+d_fz,tauz:tauz0+d_tauz}

simptrig={sin(d_theta1):1,sin(d_theta2):1,sin(d_theta3):1,sin(d_theta4):1,sin(d_omega1):1,sin(d_omega2):1,sin(d_omega3):1,sin(d_omega4):1,sin(d_psi1):1,sin(d_psi2):1,sin(d_psi3):1,sin(d_psi4):1,cos(d_theta1):1,cos(d_theta2):1,cos(d_theta3):1,cos(d_theta4):1,cos(d_omega1):1,cos(d_omega2):1,cos(d_omega3):1,cos(d_omega4):1,cos(d_psi1):1,cos(d_psi2):1,cos(d_psi3):1,cos(d_psi4):1}
simpdelta={d_theta1:d_theta1*sigma,d_theta2:d_theta2*sigma,d_theta3:d_theta3*sigma,d_theta4:d_theta4*sigma,d_omega1:d_omega1*sigma,d_omega2:d_omega2*sigma,d_omega3:d_omega3*sigma,d_omega4:d_omega4*sigma,d_psi1:d_psi1*sigma,d_psi2:d_psi2*sigma,d_psi3:d_psi3*sigma,d_psi4:d_psi4*sigma}
"""

'\nstates={theta1: theta1_0 + d_theta1, theta2: theta2_0 + d_theta2, theta3: theta3_0 + d_theta3, theta4: theta4_0 + d_theta4, diff(theta1): omega10+ d_omega1, diff(theta2): omega20+ d_omega2, diff(theta3): -omega20 - d_omega2, diff(theta4): omega40+ d_omega4} \nstates\n#States = [ seq( (theta[i](t) = theta[i[0]] + delta[theta[i]](t) ),i=1..4),seq( ( diff(theta[i](t),t) = omega[i[0]] + delta[omega[i]](t) ),i=1..2 ), diff(theta[3](t),t) = -omega_20 - deltaomega2,diff(theta[4](t),t) = omega[4[0]] + delta[omega[4]](t)  ];\ninputs={fx:fx0+d_fx,fy:fy0+d_fy,fz:fz0+d_fz,tauz:tauz0+d_tauz}\n\nsimptrig={sin(d_theta1):1,sin(d_theta2):1,sin(d_theta3):1,sin(d_theta4):1,sin(d_omega1):1,sin(d_omega2):1,sin(d_omega3):1,sin(d_omega4):1,sin(d_psi1):1,sin(d_psi2):1,sin(d_psi3):1,sin(d_psi4):1,cos(d_theta1):1,cos(d_theta2):1,cos(d_theta3):1,cos(d_theta4):1,cos(d_omega1):1,cos(d_omega2):1,cos(d_omega3):1,cos(d_omega4):1,cos(d_psi1):1,cos(d_psi2):1,cos(d_psi3):1,cos(d_psi4):1}\nsimpdelta={d_theta1:d_theta1

In [114]:
#E1= Mdf[1].subs({q[0]:0,q_dot[0]:0,q_ddot[0]:0,q_ddot[2]:-q_ddot[1]})
#simplify(E1)
#sols = solve(E1,q_ddot[1])
#sols[0].factor().evalf()

In [115]:
#E1= Mdf[3].subs({q[0]:0,q_dot[0]:0,q_ddot[0]:0,q_ddot[2]:-q_ddot[1]})
#simplify(E1)
#sols = solve(E1,q_ddot[3])
#sols[0].factor().evalf()

In [116]:
#Trim_value={fx:fx0,fy:fy0,fz:fz0,tauz:tauz0,theta1_0:0,theta2_0:0,theta3_0:-(pi/2),theta4_0:0,omega10:0,omega20:0,omega30:0,omega40:0,psi10:0,psi20:0,psi30:0,psi40:0}
"""
Trim={fx:fx0,fy:fy0,fz:fz0,tauz:tauz0,theta1:theta1_0,theta2:theta2_0,theta3:-theta2_0,theta4:theta4_0,diff(theta1,t):omega10,diff(theta2,t):omega20,diff(theta3,t):omega30,diff(theta4,t):omega40,diff(diff(theta1,t),t):psi10,diff(diff(theta2,t),t):psi20,diff(diff(theta3,t),t):psi30,diff(diff(theta4,t),t):psi40}

Trim_value={theta1_0: 0.00,theta2_0: 0.00,theta3_0: -(pi/2),theta4_0: 0.00,omega10: 0.00,omega20: 0.00,omega30: 0.00,omega40: 0.00,psi10: 0.00,psi20: 0.00,psi30: 0.00,psi40: 0.00}
Equi_eq=Mdf.subs(Trim)
E=Equi_eq.subs(Trim_value)
"""
#sols = solve(E)
#sols[0].factor().evalf()
#sols

'\nTrim={fx:fx0,fy:fy0,fz:fz0,tauz:tauz0,theta1:theta1_0,theta2:theta2_0,theta3:-theta2_0,theta4:theta4_0,diff(theta1,t):omega10,diff(theta2,t):omega20,diff(theta3,t):omega30,diff(theta4,t):omega40,diff(diff(theta1,t),t):psi10,diff(diff(theta2,t),t):psi20,diff(diff(theta3,t),t):psi30,diff(diff(theta4,t),t):psi40}\n\nTrim_value={theta1_0: 0.00,theta2_0: 0.00,theta3_0: -(pi/2),theta4_0: 0.00,omega10: 0.00,omega20: 0.00,omega30: 0.00,omega40: 0.00,psi10: 0.00,psi20: 0.00,psi30: 0.00,psi40: 0.00}\nEqui_eq=Mdf.subs(Trim)\nE=Equi_eq.subs(Trim_value)\n'

In [117]:
"""
Eq=[[0],[0],[0],[0]]
for i in range(4):
    TE0=Mdf[i].subs(states)
    TE0=TE0.subs(inputs)
    TE1=trigsimp(TE0) 
    TE2=TE1.subs(simptrig)
    TE3=TE2.subs(simpdelta)
    TE4= TE3.subs({sigma**9:0})
    TE4= TE4.subs({sigma**8:0})
    TE4= TE4.subs({sigma**7:0})
    TE4= TE4.subs({sigma**6:0})
    TE4= TE4.subs({sigma**5:0})
    TE4= TE4.subs({sigma**4:0})
    TE4= TE4.subs({sigma**3:0})
    TE4= TE4.subs({sigma**2:0})
    TE4= TE4.expand()
    TE5= TE4.subs({sigma:1})
    # Eliminação dos termos que advem do equilibrio

    TE6= expand(TE5-Equi_eq[i])
    Eq[i]=TE6
"""

'\nEq=[[0],[0],[0],[0]]\nfor i in range(4):\n    TE0=Mdf[i].subs(states)\n    TE0=TE0.subs(inputs)\n    TE1=trigsimp(TE0) \n    TE2=TE1.subs(simptrig)\n    TE3=TE2.subs(simpdelta)\n    TE4= TE3.subs({sigma**9:0})\n    TE4= TE4.subs({sigma**8:0})\n    TE4= TE4.subs({sigma**7:0})\n    TE4= TE4.subs({sigma**6:0})\n    TE4= TE4.subs({sigma**5:0})\n    TE4= TE4.subs({sigma**4:0})\n    TE4= TE4.subs({sigma**3:0})\n    TE4= TE4.subs({sigma**2:0})\n    TE4= TE4.expand()\n    TE5= TE4.subs({sigma:1})\n    # Eliminação dos termos que advem do equilibrio\n\n    TE6= expand(TE5-Equi_eq[i])\n    Eq[i]=TE6\n'

In [118]:
#Trim_value={fx:fx0,fy:fy0,fz:fz0,tauz:tauz0,theta1_0:0,theta2_0:0,theta3_0:-(pi/2),theta4_0:0,omega10:0,omega20:0,omega30:0,omega40:0,psi10:0,psi20:0,psi30:0,psi40:0}
#Eq[1].subs(Trim_value).evalf()